In [4]:
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, DataCollatorForLanguageModeling, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [5]:
ds = load_dataset("zefang-liu/phishing-email-dataset", split="train")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [7]:
def tokenization(batch):
    texts = [str(text) for text in batch["Email Text"]]
    return tokenizer(texts, padding = True, truncation = True)

token_ds = ds.map(tokenization, batched = True)

In [8]:
def create_label(batch):
    label_map = {"Phishing Email": 1, "Safe Email": 0}
    batch["label"] = label_map[batch["Email Type"]]
    return batch

final_ds = token_ds.map(create_label)

In [9]:
final_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [10]:
seed = 42
shuffled_ds = final_ds.shuffle(seed)

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
dataloader = DataLoader(shuffled_ds, batch_size=8, collate_fn=data_collator)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model.to(device)
model.train()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
epochs = 5
for epoch in range(epochs):
    for i, batch in enumerate(tqdm(dataloader, total=5)):
        if i == 5:
            break
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

 20%|████████████▌                                                  | 1/5 [00:17<01:10, 17.57s/it]

loss: 0.6868773102760315


 20%|████████████▌                                                  | 1/5 [00:13<00:55, 13.99s/it]

loss: 0.6282207369804382


 20%|████████████▌                                                  | 1/5 [00:14<00:56, 14.19s/it]

...

 20%|████████████▌                                                  | 1/5 [00:15<01:00, 15.11s/it]

loss: 0.5855942368507385


 20%|████████████▌                                                  | 1/5 [00:14<00:57, 14.25s/it]

loss: 0.5426371097564697


100%|███████████████████████████████████████████████████████████████| 5/5 [01:14<00:00, 14.90s/it]


In [13]:
model.save_pretrained("phish_detect_V1")
tokenizer.save_pretrained("phish_detect_V1")

('phish_detect_V1/tokenizer_config.json',
 'phish_detect_V1/special_tokens_map.json',
 'phish_detect_V1/vocab.txt',
 'phish_detect_V1/added_tokens.json',
 'phish_detect_V1/tokenizer.json')

In [14]:
# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("phish_detect_V1").to(device)
tokenizer = AutoTokenizer.from_pretrained("phish_detect_V1")

model.eval()  # Set model to evaluation mode


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [67]:
def predict_email(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
    
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]
        
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Phishing Email" if prediction == 1 else "Safe Email"

email_text = "Congratulations! You've won a free iPhone. Click the link to claim your prize."
print(predict_email(email_text))  # Should return "Phishing Email" if trained well


Phishing Email


In [68]:
import pandas as pd

In [69]:
test_ds = pd.read_csv("test_dataset.csv")

In [70]:
test_ds.head()

,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1


In [72]:
test = Dataset.from_pandas(test_ds)

In [74]:
def tokenize_test_data(batch):
    texts = [str(text) for text in batch["body"]]
    return tokenizer(texts, padding = True, truncation = True)

token_test = test.map(tokenize_test_data, batched = True)

Map:   0%|          | 0/39154 [00:00<?, ? examples/s]

In [75]:
token_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [78]:
test_dataloader = DataLoader(token_test, batch_size=8, collate_fn=data_collator)

In [82]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Predicting", total=len(test_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)

label_map = {1: "Phishing Email", 0: "Safe Email"}
predicted_labels = [label_map[pred] for pred in predictions]

test_df = token_test.to_pandas()
test_df["Predicted Label"] = predicted_labels

print(test_df[["body", "label", "Predicted Label"]].head())

Predicting: 100%|█████████████████████████| 4895/4895 [2:53:50<00:00,  2.13s/it]


                                                body  label Predicted Label
0  Buck up, your troubles caused by small dimensi...      1  Phishing Email
1  \nUpgrade your sex and pleasures with these te...      1  Phishing Email
2  >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...      1      Safe Email
3  Would anyone object to removing .so from this ...      0      Safe Email
4  \nWelcomeFastShippingCustomerSupport\nhttp://7...      1  Phishing Email


In [83]:
from sklearn.metrics import accuracy_score, classification_report

In [91]:
true_labels = test_df["label"].tolist()
label_map_r = {"Phishing Email": 1, "Safe Email": 0}
pred_labels_num = [label_map_r[label] for label in predicted_labels]
accuracy = accuracy_score(true_labels, pred_labels_num)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(true_labels, pred_labels_num, target_names=["Safe Email", "Phishing Email"]))

Accuracy: 0.8684
                precision    recall  f1-score   support

    Safe Email       0.79      0.95      0.87     17312
Phishing Email       0.96      0.80      0.87     21842

      accuracy                           0.87     39154
     macro avg       0.87      0.88      0.87     39154
  weighted avg       0.88      0.87      0.87     39154

